In [1]:
import requests
from bs4 import BeautifulSoup


In [3]:
url = "https://uk.wikipedia.org/wiki/%D0%A1%D0%B5%D1%80%D0%B5%D0%B4%D0%BD%D1%8F_%D0%B7%D0%B0%D1%80%D0%BE%D0%B1%D1%96%D1%82%D0%BD%D0%B0_%D0%BF%D0%BB%D0%B0%D1%82%D0%B0_%D0%B2_%D0%A3%D0%BA%D1%80%D0%B0%D1%97%D0%BD%D1%96"
response = requests.get(url)
html = response.text


In [4]:
soup = BeautifulSoup(html, 'html.parser')


In [9]:
tables = soup.find_all('table')
print(len(tables))


3


In [6]:
for i in range(3):
    table = tables[i]
    caption = table.find('caption')
    print(f"Table {i + 1}: {caption.text if caption else 'No caption'}")


Table 1: No caption
Table 2: No caption
Table 3: No caption


In [10]:
third_table = tables[2]

Виведіть перші 5 рядків таблиці.

In [11]:

rows = third_table.find_all('tr')


for row in rows[:5]:
    cells = row.find_all('td')
    for cell in cells:
        print(cell.get_text(), end='\t')
    print()



1992
	5832,88▲
	28,04▼
	1015,67▲
	—
	
1993
	163533▲
	36,03▲
	430,42▼
	—
	
1994
	1426343▲
	44,99▲
	372,76▼
	—
	
1995
	7304000▲
	49,53▲
	201,76▼
	—
	


Визначте кількість рядків і стовпчиків за допомогою атрибуту shape

In [12]:
import pandas as pd

In [13]:
html_table = str(third_table)

In [14]:
df = pd.read_html(html_table)[0]

In [15]:
rows, columns = df.shape
print(f"Кількість рядків: {rows}")
print(f"Кількість стовпчиків: {columns}")

Кількість рядків: 32
Кількість стовпчиків: 5


Замінити пропущені значення  «-» в таблиці на NaN

In [17]:
df.replace('-', pd.NA, inplace=True)
print(df)

      Рік В національній валюті В доларах США  \
0    1992              5832,88▲        28,04▼   
1    1993               163533▲        36,03▲   
2    1994              1426343▲        44,99▲   
3    1995              7304000▲        49,53▲   
4    1996               125,23▼        68,45▲   
5    1997               143,09▲        76,86▲   
6    1998               152,83▲        62,39▼   
7    1999               177,39▲        42,95▼   
8    2000               231,04▲        42,47▼   
9    2001               311,62▲        58,01▲   
10   2002               375,98▲        70,59▲   
11   2003               462,58▲        86,74▲   
12   2004               590,53▲       111,02▲   
13   2005               806,14▲       157,30▲   
14   2006              1042,88▲       206,51▲   
15   2007              1352,75▲       267,87▲   
16   2008              1808,92▲       343,43▲   
17   2009              1909,25▲       245,05▼   
18   2010              2246,75▲       283,12▲   
19   2011           

Визначити тип стовпців за допомогою dtypes.

In [18]:
column_types = df.dtypes
print(column_types)

Рік                                           object
В національній валюті                         object
В доларах США                                 object
Кількість кг хліба[20] на середню зарплату    object
Кількість БігМаків на середню зарплату        object
dtype: object


Замінити тип нечислових даних на числові.

In [23]:
columns_to_convert = ['Рік']

for column in columns_to_convert:
    df[column] = pd.to_numeric(df[column], errors='coerce')

Порахуйте частку пропусків в кожній колонці (за допомогою методів isnull , sum, len)

In [24]:
missing_ratio = {}


for column in df.columns:
    total_count = len(df)
    missing_count = df[column].isnull().sum()
    ratio = (missing_count / total_count) * 100
    missing_ratio[column] = ratio

for column, ratio in missing_ratio.items():
    print(f'Стовпець "{column}": {ratio:.2f}% пропусків')

Стовпець "Рік": 6.25% пропусків


Замініть всі пропущені значення на середні значення по (стовпцю).



In [31]:
df = df.fillna(df.mean())